In [19]:
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score

In [20]:
train_data=pd.read_csv('/home/dugrarao/Documents/NLP/Tweetproject/train_2kmZucJ.csv')


In [21]:
#cleaning train data
for i in range(len(train_data['tweet'])):
    txt = train_data.loc[i]["tweet"]
    txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace @user tags
    txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
    txt=re.sub("[^a-zA-Z]", " ",txt)#replace hashtags
    train_data.at[i,"tweet"]=txt


In [22]:
#removing words that are less than 3
train_data['tweet']=train_data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


In [23]:
#Tokenizing
tokenized_train_data=train_data['tweet'].apply(lambda x: x.split())


In [24]:
train_data['tweet'] = tokenized_train_data.apply(lambda x:' '.join([lem.lemmatize(i) for i in x])) # lemmatizing


In [25]:
#bagofwords
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer_train_data = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow_train_data = bow_vectorizer_train_data.fit_transform(train_data['tweet'])
bow_train_data

<7920x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 49661 stored elements in Compressed Sparse Row format>

In [26]:
#Tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer_train_data = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf_train_data = tfidf_vectorizer_train_data.fit_transform(train_data['tweet'])
tfidf_train_data

<7920x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 49661 stored elements in Compressed Sparse Row format>

In [27]:
#splitting train data
x_train, x_test, y_train, y_test = train_test_split(tfidf_train_data,train_data["label"], test_size = 0.2, random_state = 42)
x_train,x_test

(<6336x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 39734 stored elements in Compressed Sparse Row format>,
 <1584x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 9927 stored elements in Compressed Sparse Row format>)

In [28]:
#Building Random Classifier model
rf=RandomForestClassifier(criterion='gini',random_state=0)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [29]:
train_data_predicted_val=rf.predict(x_test)
train_data_predicted_val

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
f1_score(y_test,train_data_predicted_val)

0.75

In [31]:
confusion_matrix(y_test,train_data_predicted_val)

array([[1089,   63],
       [ 135,  297]])

In [32]:
accuracy_score(y_test,train_data_predicted_val)

0.875

In [33]:
#For test data
test_data=pd.read_csv('test_oJQbWVk.csv')


In [34]:
#Cleaning test data
for i in range(len(test_data['tweet'])):
    txt = test_data.loc[i]["tweet"]
    txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace username-tags
    txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
    txt=re.sub("[^a-zA-Z]", " ",txt)#replace hashtags
    test_data.at[i,"tweet"]=txt
#removing words that are less than 3
test_data['tweet']=test_data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#Tokenizing
tokenized_test_data=test_data['tweet'].apply(lambda x: x.split())
test_data['tweet'] = tokenized_test_data.apply(lambda x:' '.join([lem.lemmatize(i) for i in x])) # lemmatizing


In [35]:
#Bagofwords
bow_vectorizer_test_data = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow_test_data = bow_vectorizer_test_data.fit_transform(test_data['tweet'])


In [36]:
#Tf-idf

tfidf_vectorizer_test_data = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf_test_data = tfidf_vectorizer_test_data.fit_transform(test_data['tweet'])
tfidf_test_data

<1953x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 12191 stored elements in Compressed Sparse Row format>

In [37]:
rf=RandomForestClassifier(n_estimators=200,criterion='gini',random_state=0)
rf.fit(tfidf_train_data, train_data['label'])
test_data_predicted_val=rf.predict(tfidf_test_data)

In [38]:
#Predictions to csv
final_result = pd.DataFrame({'id':test_data['id'],'label':test_data_predicted_val})
final_result.to_csv('RandomForestOutput.csv',index=False)

